In [1]:
import boto3

def run_query(query, database, s3_output):
    client_athena = boto3.client('athena', region_name = 'us-east-1')
    response = client_athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    return response

In [2]:
import pandas as pd
import pyarrow
import pandasglue as pg
pd.__version__

'0.24.1'

In [4]:
%%time
# FORMAT PARQUET FROM PANDAS
client_s3 = boto3.client('s3')
s3 = boto3.resource('s3')
bucket_name = 'attemp-glue'
bucket_csv = s3.Bucket(bucket_name)
marca = 'gestion.pe'

key_input = '{0}/2018/12'.format(marca)
bucket_output = 'athena-attemp'

# CREO UN DATAFRAME CON TODOS LOS DIAS DEL MES
list_df = []
for obj in bucket_csv.objects.filter(Prefix=key_input):
    key = obj.key
    url_up = 'https://s3-us-west-1.amazonaws.com/{}/{}'.format(bucket_name, key)
    list_df.append(
        pd.read_csv(url_up)
    )

all_data_df = pd.concat(list_df)
del list_df

CPU times: user 735 ms, sys: 97.7 ms, total: 833 ms
Wall time: 18.2 s


In [5]:
all_data_df.head()

,dia,page_avg_time,mes
0,2018-12-18,61,12
1,2018-12-18,44,12
2,2018-12-18,0,12
3,2018-12-18,67,12
4,2018-12-18,15,12


In [6]:
# PASO EL CONTENIDO DE MI DATAFRAME A FORMATO 
# PARQUET PARTICIONADO POR LA COLUMNA DIA EN COMPRESION SNAPPY

all_data_df.to_parquet(
    marca, engine='auto', compression='snappy', index=None, partition_cols=['dia']
)

In [7]:
# SUBO LOS ARCHIVOS EN FORMATO PARQUET A UN BUCKET
import glob
path_dir_parquet = '/home/rmg/STORAGE/tests_courses/practice/ATHENA/gestion.pe'

for filename in glob.iglob(path_dir_parquet + '/**/*.parquet', recursive=True):
    client_s3.upload_file(
        filename, bucket_output, filename.split('ATHENA/')[-1].replace('dia=', 'date_part='),
        ExtraArgs={'ACL': 'public-read'}
    )
# OJO: Se cambia la particion dia por el nombre date_part para que sea compatible con HIVE

In [23]:
%%time
import time

bucket_db = 'athena-db-table'

s3_input = 's3://{0}/{1}/'.format(bucket_output, marca)
s3_ouput = 's3://{0}'.format(bucket_db)

print(s3_input)
print(s3_ouput)
database = 'athenatest'
table = 'charbeat2'

create_database = "CREATE DATABASE IF NOT EXISTS {0};".format(database)
res_1 = run_query(create_database, database, s3_ouput)
res_1

s3://athena-attemp/gestion.pe/
s3://athena-db-table
CPU times: user 57.8 ms, sys: 7.11 ms, total: 64.9 ms
Wall time: 877 ms


In [24]:
database, table, s3_input

('athenatest', 'charbeat2', 's3://athena-attemp/gestion.pe/')

In [25]:
create_table = \
"""
CREATE EXTERNAL TABLE IF NOT EXISTS {0}.{1}(  
dia STRING,
mes INT,
page_avg_time INT
)
PARTITIONED BY (date_part STRING)
STORED AS PARQUET
LOCATION '{2}';
""".format(database, table, s3_input)
print(create_table)

res = run_query(create_table, database, s3_ouput)
print(res)


CREATE EXTERNAL TABLE IF NOT EXISTS athenatest.charbeat2(  
dia STRING,
mes INT,
page_avg_time INT
)
PARTITIONED BY (date_part STRING)
STORED AS PARQUET
LOCATION 's3://athena-attemp/gestion.pe/';

{'QueryExecutionId': 'b86f7951-3f73-44d6-9162-9a0d7443a190', 'ResponseMetadata': {'RequestId': '19557d11-addc-467a-8872-07358a142066', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Wed, 27 Mar 2019 19:41:25 GMT', 'x-amzn-requestid': '19557d11-addc-467a-8872-07358a142066', 'content-length': '59', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [26]:
create_execution = res.get('QueryExecutionId', '')
create_execution

'b86f7951-3f73-44d6-9162-9a0d7443a190'

In [27]:
client_athena = boto3.client('athena', region_name = 'us-east-1')
response = client_athena.get_query_execution(QueryExecutionId=create_execution)
response

{'QueryExecution': {'QueryExecutionId': 'b86f7951-3f73-44d6-9162-9a0d7443a190',
  'Query': "CREATE EXTERNAL TABLE IF NOT EXISTS athenatest.charbeat2(  \ndia STRING,\nmes INT,\npage_avg_time INT\n)\nPARTITIONED BY (date_part STRING)\nSTORED AS PARQUET\nLOCATION 's3://athena-attemp/gestion.pe/'",
  'StatementType': 'DDL',
  'ResultConfiguration': {'OutputLocation': 's3://athena-db-table/b86f7951-3f73-44d6-9162-9a0d7443a190.txt'},
  'QueryExecutionContext': {'Database': 'athenatest'},
  'Status': {'State': 'SUCCEEDED',
   'SubmissionDateTime': datetime.datetime(2019, 3, 27, 14, 41, 25, 814000, tzinfo=tzlocal()),
   'CompletionDateTime': datetime.datetime(2019, 3, 27, 14, 41, 26, 232000, tzinfo=tzlocal())},
  'Statistics': {'EngineExecutionTimeInMillis': 340, 'DataScannedInBytes': 0}},
 'ResponseMetadata': {'RequestId': 'aafefed8-8e37-4c39-aaec-f860f560f848',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 27 Mar 2019 19:41:44 GMT',

In [28]:
msck_table = 'MSCK REPAIR TABLE {0}.{1};'.format(database, table)
res_msck = run_query(msck_table, database, s3_ouput)
res_msck

{'QueryExecutionId': '78a1aba7-6f94-4424-bb45-31027e5f8999',
 'ResponseMetadata': {'RequestId': '89b36f42-832b-4573-a569-53dff507b745',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 27 Mar 2019 19:42:08 GMT',
   'x-amzn-requestid': '89b36f42-832b-4573-a569-53dff507b745',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [29]:
msck_execution = res_msck.get('QueryExecutionId', '')
msck_execution

'78a1aba7-6f94-4424-bb45-31027e5f8999'

In [30]:
response_msck = client_athena.get_query_execution(QueryExecutionId=msck_execution)
response_msck

{'QueryExecution': {'QueryExecutionId': '78a1aba7-6f94-4424-bb45-31027e5f8999',
  'Query': 'MSCK REPAIR TABLE athenatest.charbeat2',
  'StatementType': 'DDL',
  'ResultConfiguration': {'OutputLocation': 's3://athena-db-table/78a1aba7-6f94-4424-bb45-31027e5f8999.txt'},
  'QueryExecutionContext': {'Database': 'athenatest'},
  'Status': {'State': 'SUCCEEDED',
   'SubmissionDateTime': datetime.datetime(2019, 3, 27, 14, 42, 8, 739000, tzinfo=tzlocal()),
   'CompletionDateTime': datetime.datetime(2019, 3, 27, 14, 42, 12, 199000, tzinfo=tzlocal())},
  'Statistics': {'EngineExecutionTimeInMillis': 3176,
   'DataScannedInBytes': 0}},
 'ResponseMetadata': {'RequestId': 'ec6a31fd-b0a3-4a46-8725-c715d1b1637e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 27 Mar 2019 19:42:13 GMT',
   'x-amzn-requestid': 'ec6a31fd-b0a3-4a46-8725-c715d1b1637e',
   'content-length': '1132',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [31]:
query_all = "SELECT * FROM {}.{}".format(database, table)
res_query = run_query(query_all, database, s3_ouput)
res_query

{'QueryExecutionId': 'c12e33af-86c7-4cec-b5d9-fc22bcf54f0c',
 'ResponseMetadata': {'RequestId': '9318ebbb-b6e1-4ad7-90a0-146d2d58774d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 27 Mar 2019 19:42:21 GMT',
   'x-amzn-requestid': '9318ebbb-b6e1-4ad7-90a0-146d2d58774d',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [32]:
query_execution = res_query.get('QueryExecutionId', '')
query_execution

'c12e33af-86c7-4cec-b5d9-fc22bcf54f0c'

In [33]:
response_query = client_athena.get_query_execution(QueryExecutionId=query_execution)
response_query

{'QueryExecution': {'QueryExecutionId': 'c12e33af-86c7-4cec-b5d9-fc22bcf54f0c',
  'Query': 'SELECT * FROM athenatest.charbeat2',
  'StatementType': 'DML',
  'ResultConfiguration': {'OutputLocation': 's3://athena-db-table/c12e33af-86c7-4cec-b5d9-fc22bcf54f0c.csv'},
  'QueryExecutionContext': {'Database': 'athenatest'},
  'Status': {'State': 'SUCCEEDED',
   'SubmissionDateTime': datetime.datetime(2019, 3, 27, 14, 42, 22, 511000, tzinfo=tzlocal()),
   'CompletionDateTime': datetime.datetime(2019, 3, 27, 14, 42, 25, 561000, tzinfo=tzlocal())},
  'Statistics': {'EngineExecutionTimeInMillis': 2719,
   'DataScannedInBytes': 201876}},
 'ResponseMetadata': {'RequestId': '4839499d-322d-4b15-b6a9-f4344e8505bc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 27 Mar 2019 19:42:30 GMT',
   'x-amzn-requestid': '4839499d-322d-4b15-b6a9-f4344e8505bc',
   'content-length': '1150',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [36]:
result_url = response_query.get('QueryExecution', {}).get('ResultConfiguration', {}).get('OutputLocation', '')
result_url

's3://athena-db-table/c12e33af-86c7-4cec-b5d9-fc22bcf54f0c.csv'

In [37]:
df_result = pd.read_csv(result_url)
df_result.head()

,dia,mes,page_avg_time,date_part
0,NaN,12,0,2018-12-31
1,NaN,12,10,2018-12-31
2,NaN,12,17,2018-12-31
3,NaN,12,79,2018-12-31
4,NaN,12,14,2018-12-31


In [38]:
df_result.shape

(178242, 4)